# Wine quality prediction using Machine Learning

The original source code and explanations can be found in the following link

https://www.geeksforgeeks.org/wine-quality-prediction-machine-learning/?ref=lbp

Here we will predict the quality of wine on the basis of given features. We use the wine quality dataset available on Internet for free. This dataset has some fundamental features which may the quality of the wine. We will predict the quality of the wine using different machine learning algorithms.

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')